# Выбор числа соседей

## 1. Загрузите выборку Wine по адресу https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data (файл также приложен к этому заданию)  
Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), признаки — в столбцах со второго по последний. Более подробно о сути признаков можно прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names, приложенный к заданию)

In [8]:
import pandas as pd
import numpy as np
columns = ['Grape variety', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 
           'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
data = pd.read_csv('./data/wine.data', sep=",", names=columns)  
print(len(data))
data.head()

178


,Grape variety,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
import sklearn
clf = sklearn.neighbors.KNeighborsClassifier

## 3. Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). Создайте генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным параметром random_state=42. В качестве меры качества используйте долю верных ответов (accuracy).  
Создается генератор разбиений sklearn.model_selection.KFold, который задает набор разбиений на обучение и валидацию. Число блоков в кросс-валидации определяется параметром n_splits. Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания достаточно передать генератору KFold параметр shuffle=True.

In [36]:
import sklearn
from sklearn.model_selection import KFold

print(sklearn.__version__)

split_generator = KFold(n_splits=5, shuffle=True, random_state=42)
X = data[columns[1:]]
# y = data[columns[:1]] так нельзя 
y = data['Grape variety']

0.20.3


## 4. Найдите точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? Данные результаты и будут ответами на вопросы 1 и 2.  
Вычислить качество на всех разбиениях можно при помощи функции sklearn.model_selection.cross_val_score. В качестве параметра estimator передается классификатор, в качестве параметра cv — генератор разбиений с предыдущего шага. С помощью параметра scoring можно задавать меру качества, по умолчанию в задачах классификации используется доля верных ответов (accuracy). Результатом является массив, значения которого нужно усреднить.

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

cvs_1 = np.ndarray(shape=50, dtype=float)
for i in range(1, 50):
    clf = KNeighborsClassifier(i)
    cvs = cross_val_score(estimator=clf, X=X, y=y, cv=split_generator, scoring='accuracy')
    _ =cvs.mean()
    print(f"iteration: {i}, cvs: {_}")
    cvs_1[i-1] = _

iteration: 1, cvs: 0.7304761904761905
iteration: 2, cvs: 0.6625396825396825
iteration: 3, cvs: 0.7082539682539682
iteration: 4, cvs: 0.6577777777777778
iteration: 5, cvs: 0.6746031746031746
iteration: 6, cvs: 0.6742857142857144
iteration: 7, cvs: 0.68
iteration: 8, cvs: 0.6799999999999999
iteration: 9, cvs: 0.7023809523809523
iteration: 10, cvs: 0.6801587301587302
iteration: 11, cvs: 0.7025396825396826
iteration: 12, cvs: 0.6966666666666667
iteration: 13, cvs: 0.6909523809523809
iteration: 14, cvs: 0.6793650793650794
iteration: 15, cvs: 0.701904761904762
iteration: 16, cvs: 0.6795238095238096
iteration: 17, cvs: 0.7015873015873015
iteration: 18, cvs: 0.6795238095238095
iteration: 19, cvs: 0.6793650793650793
iteration: 20, cvs: 0.6909523809523809
iteration: 21, cvs: 0.7019047619047619
iteration: 22, cvs: 0.6965079365079364
iteration: 23, cvs: 0.7020634920634921
iteration: 24, cvs: 0.7076190476190476
iteration: 25, cvs: 0.7019047619047619
iteration: 26, cvs: 0.6965079365079365
iteration:

In [57]:
print(f"index: {cvs_1.argmax()} value: {cvs_1[cvs_1.argmax()]} {round(cvs_1.max(), 2)}") 

index: 0 value: 0.7304761904761905 0.73
